# 0) vérifier [sc] et [spark]

In [1]:
# le spark context détermine les ressources 
# disponibles pour l'application
assert sc==sc

In [2]:
# la variable spark permet de gérer les
# dataframes (dépend de pyspark.sql)
assert spark==spark

In [3]:
spark

# -1) Création de fonctions distribuable ...

In [88]:
# on fait une opération associative et commutative (afin qu'elle soit distruable)
def somme_de_valeurs(v1, v2):
    return v1 + v2
def concatenation_de_liste (liste1, liste2):  
    if type(liste1)==type([]):
        return sum(liste1)+sum(liste2)
    else :
        return "pbm avec  %s"%type(liste1)
def moyenne_de_liste( x, y): 
    if type(liste1)==type([]):
        liste_concatennees = x + y
        return np.mean( liste_concatennees )
    else :
        return "pbm avec  %s"%type(liste1)
    
def max_de_liste( x, y): 
    if type(liste1)==type([]):
        liste_concatennees = x + y
        return np.max( liste_concatennees )
    else :
        return "pbm avec  %s"%type(liste1)    

# 1) loader un fichier en mémoire

In [38]:
!ls /home/spark/spark/velib/pyspark/cycliste*

/home/spark/spark/velib/pyspark/cycliste_cyclistes.csv
/home/spark/spark/velib/pyspark/cycliste_debug.csv.zip
/home/spark/spark/velib/pyspark/cyclistes.csv


# 1.1) dézipper cycliste_debug (sur toutes les machines du cluster !)
-> ! unzip /home/spark/spark/velib/pyspark/cycliste_debug.csv.zip

In [49]:
! unzip /home/spark/spark/velib/pyspark/cycliste_debug.csv.zipN

Archive:  /home/spark/spark/velib/pyspark/cycliste_debug.csv.zip
replace cycliste_debug.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [54]:
!ls -l cycliste*

-rw-rw-r-- 1 spark spark   282378 Dec 11 22:09 cycliste_cyclistes.csv
-rw-r--r-- 1 spark spark 21639712 May 30  2017 cycliste_debug.csv
-rw-rw-r-- 1 spark spark  2318793 Dec 12 01:05 cycliste_debug.csv.zip
-rw-rw-r-- 1 spark spark  2333044 Dec 11 22:09 cyclistes.csv


In [55]:
path  = "hdfs://spark52:8020/cycliste_debug_2.csv"
# si on prend un répertoire, alors la rdd comprendra tous les fichiers
# du répertoire
path = "/home/spark/spark/velib/pyspark/cycliste_debug.csv" 

In [56]:
rdd  = sc.textFile(path)
logs = rdd

In [57]:
logs.count()

247574

## 1.1) essayer de comprendre le fichier

In [58]:
logs.count() # nombre de lignes

247574

In [59]:
logs.take(5) # extrait

['cycliste,n_message,time,message',
 'cycliste_aztu4,0000, 1496170615.88,station_maison = azuf3',
 'cree',
 'cycliste_aztu4,0002, 1496170615.88,station_maison nearest= aeqc6|aeohw|azum7|aeyd1|artgc|arojk|aztop|aruk0',
 'cycliste_aztu4,0001, 1496170615.88,station_travail = aeghv']

In [60]:
header = logs.take(1)
header = header[0]
header

'cycliste,n_message,time,message'

In [61]:
sans_header = logs.filter(lambda ligne: ligne != header)
assert(sans_header.count() == logs.count() -1)

#### 1.1)  nombre de virgules par lignes

In [62]:
separator       = ","
lambda_function = lambda log: len(log.split(separator))
nb_virgules     = logs.map(lambda_function)

In [63]:
nb_possible  = nb_virgules.distinct()
nb_possible.collect()

[4, 5, 1]

##### 1.2) trouver des logs pour chaque nb de virgules

In [64]:
create_pair = lambda log: (len(log.split(separator)), log)
pairedRdd   = logs.map(create_pair) 
pairedRdd.take(3)

[(4, 'cycliste,n_message,time,message'),
 (4, 'cycliste_aztu4,0000, 1496170615.88,station_maison = azuf3'),
 (1, 'cree')]

In [65]:
pairedRdd.groupByKey().take(3) # renvoie un itérateur (ouf !)

[(4, <pyspark.resultiterable.ResultIterable at 0x7fa948ec7cc0>),
 (5, <pyspark.resultiterable.ResultIterable at 0x7fa948ec7550>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7fa948ec7518>)]

In [66]:
pairedRdd.countByKey() 

defaultdict(int, {1: 2314, 4: 202465, 5: 42795})

##### 1.2.1) on affiche pour chaque nb de virgules quelques exemples

In [67]:
key_len__iterateur = pairedRdd.groupByKey().take(3)
for key_len, iterateur in key_len__iterateur:
    print ("="*20)
    print ("nb de virgules = ", key_len)
    print ("exemples : ")
    for i in range(3):
        print (iterateur.data[i])
        

nb de virgules =  4
exemples : 
cycliste_azuq6,0043, 1496170672.35,velo defaillant <0.5
cycliste_azudm,0044, 1496170674.16,self.a_la_maison = False
cycliste_azudm,0043, 1496170672.35,True a pris le velo velo_azup0_1496170613.03 sur la station azxbn
nb de virgules =  5
exemples : 
cycliste_azehn,0044, 1496170674.16,impossible de rendre sur ,ardlx ( a la maison = False)
cycliste_azyui,0044, 1496170674.16,impossible de rendre sur ,ardlx ( a la maison = False)
cycliste_azex3,0052, 1496170674.16,impossible de rendre sur ,ardlx ( a la maison = True)
nb de virgules =  1
exemples : 
cree
cree
cree


#### 1.2.2 supprimeons les "cree"

In [89]:
logs_sans_cree = logs.filter(lambda log: log !="cree")
logs_sans_cree.count()

245260

In [90]:
log_a_5_virgules = logs_sans_cree.filter(lambda log: len(log.split(separator))==5)
log_a_5_virgules.count()

42795

In [91]:
log_des_impossibles = log_a_5_virgules.filter(lambda log : "impossible" in log)
log_des_impossibles.count()

36847

#### 1.3 Elements distincts entre deux rdd => substract

In [92]:
cinq_virgules_non_impossible = log_a_5_virgules.subtract(log_des_impossibles)
cinq_virgules_non_impossible.count()

5948

In [93]:
cinq_virgules_non_impossible.take(4)

['cycliste_azif4,0006, 1496170627.16,velo defaillant <0.3, rendu a la meme station :-( ',
 'cycliste_azeqs,0008, 1496170628.52,velo defaillant <0.3, rendu a la meme station :-( ',
 'cycliste_azuic,0015, 1496170630.12,velo defaillant <0.1, rendu a la meme station :-( ',
 'cycliste_azob6,0015, 1496170630.19,velo defaillant <0.3, rendu a la meme station :-( ']

In [94]:
logs_tristes = cinq_virgules_non_impossible.filter(lambda log: ":-(" in log)
logs_tristes.count()

5948

In [95]:
logs_tristes.countApproxDistinct()

6008

### 2) cyclistes

In [96]:
cyclistes = logs.filter(lambda log: log.startswith("cycliste"))
cyclistes.count()

245260

In [97]:
sorted_cyclistes = cyclistes.sortBy(lambda x: x)
sorted_cyclistes.take(2)

['cycliste,n_message,time,message',
 'cycliste_aze12,0000, 1496170618.45,station_maison = azupx']

# Sample

In [98]:
nb_de_ligne_voulu = 10
nb_ligne_existant = sorted_cyclistes.count()
pourcentage_voulu = nb_de_ligne_voulu/float(nb_ligne_existant)
avec_remise       = False

In [99]:
sorted_cyclistes.sample(avec_remise, pourcentage_voulu).collect()

['cycliste_azew6,0075, 1496170711.89,velo pris sur station azqm0 (elem de station maison)',
 'cycliste_azil1,0010, 1496170632.06,velo pris sur station azj70 (elem de station travail)',
 'cycliste_aztdg,0032, 1496170658.59,velo rendu sur la station azuf3 ( a la maison = True) (elem de station_maison)',
 'cycliste_aztil,0083, 1496170690.23,impossible de rendre sur ,azj70 ( a la maison = True)',
 'cycliste_aztmw,0011, 1496170634.77,self.a_la_maison = True',
 'cycliste_azufv,0045, 1496170671.76,impossible de rendre sur ,aejv9 ( a la maison = True)',
 'cycliste_azup3,0017, 1496170638.41,True a pris le velo velo_aryv1_1496170612.95 sur la station arsb6']

# Extraire les logs de début et fin de trajet

In [100]:
filtre = lambda log: "velo rendu" in log or "velo pris" in log 
debut_fin_trajet = sorted_cyclistes.filter(filtre)
debut_fin_trajet.take(10)

['cycliste_aze12,0004, 1496170627.84,velo pris sur station aepf4 (elem de station maison)',
 'cycliste_aze12,0008, 1496170629.68,velo rendu sur la station azs58 ( a la maison = False) (elem de station_travail)',
 'cycliste_aze12,0011, 1496170633.47,velo pris sur station azk49 (elem de station travail)',
 'cycliste_aze12,0015, 1496170634.73,velo rendu sur la station arqh8 ( a la maison = True) (elem de station_maison)',
 'cycliste_aze12,0016, 1496170638.5,velo pris sur station arqh8 (elem de station maison)',
 'cycliste_aze12,0019, 1496170639.79,velo rendu sur la station aeyu0 ( a la maison = False) (elem de station_travail)',
 'cycliste_aze12,0020, 1496170644.37,velo pris sur station azs58 (elem de station travail)',
 'cycliste_aze12,0024, 1496170644.37,velo rendu sur la station azs58 ( a la maison = False) (elem de station_travail)',
 'cycliste_aze12,0025, 1496170646.53,velo pris sur station azk49 (elem de station travail)',
 'cycliste_aze12,0028, 1496170647.95,velo rendu sur la stati

# Extraire les heures de logs

In [101]:
def get_cycliste_et_timestamp_du_log(log):
    data      = log.split(",")
    cycliste  = data[0]
    timestamp = float(data[2])
    return (cycliste, timestamp)


In [102]:
cycliste_timestamps = debut_fin_trajet.map(get_cycliste_et_timestamp_du_log)
cycliste_timestamps.take(23)

[('cycliste_aze12', 1496170627.84),
 ('cycliste_aze12', 1496170629.68),
 ('cycliste_aze12', 1496170633.47),
 ('cycliste_aze12', 1496170634.73),
 ('cycliste_aze12', 1496170638.5),
 ('cycliste_aze12', 1496170639.79),
 ('cycliste_aze12', 1496170644.37),
 ('cycliste_aze12', 1496170644.37),
 ('cycliste_aze12', 1496170646.53),
 ('cycliste_aze12', 1496170647.95),
 ('cycliste_aze12', 1496170651.74),
 ('cycliste_aze12', 1496170653.28),
 ('cycliste_aze12', 1496170657.22),
 ('cycliste_aze12', 1496170657.22),
 ('cycliste_aze12', 1496170660.07),
 ('cycliste_aze12', 1496170661.29),
 ('cycliste_aze12', 1496170665.3),
 ('cycliste_aze12', 1496170666.58),
 ('cycliste_aze12', 1496170670.43),
 ('cycliste_aze12', 1496170671.67),
 ('cycliste_aze12', 1496170675.56),
 ('cycliste_aze12', 1496170676.85),
 ('cycliste_aze12', 1496170680.89)]

# grouper les durées par cycliste

In [103]:
timestamps_par_cycliste = cycliste_timestamps.groupByKey()

# Calculer les délais entre deux timestamp

In [104]:
def diff_timestamp(liste):
    """
    Retourne une liste de delta !
    """
    timestamps = liste
    r          = []
    values     = []
    for indice, valeur in enumerate(sorted(timestamps)):
        values.append(float(valeur))
        if indice>0:
            delta = values[indice] - values[indice-1]
            r.append(delta)
    return r

In [105]:
delais_par_cyclistes = timestamps_par_cycliste.mapValues(diff_timestamp)
delais_par_cyclistes.take(5) # tableau de tuples comprenant une liste

[('cycliste_aze12',
  [1.8400001525878906,
   3.7899999618530273,
   1.2599999904632568,
   3.7699999809265137,
   1.2899999618530273,
   4.579999923706055,
   0.0,
   2.1600000858306885,
   1.4200000762939453,
   3.7899999618530273,
   1.5399999618530273,
   3.940000057220459,
   0.0,
   2.8499999046325684,
   1.2200000286102295,
   4.009999990463257,
   1.2799999713897705,
   3.8500001430511475,
   1.2400000095367432,
   3.8899998664855957,
   1.2899999618530273,
   4.0400002002716064,
   1.3799998760223389,
   4.049999952316284,
   1.2999999523162842,
   3.9800000190734863,
   1.2800002098083496,
   3.8999998569488525,
   0.0,
   2.7699999809265137,
   0.0,
   2.559999942779541,
   1.3400001525878906,
   3.950000047683716,
   1.2799999713897705,
   4.019999980926514,
   1.3399999141693115,
   4.1000001430511475,
   1.3499999046325684,
   3.940000057220459,
   1.3499999046325684]),
 ('cycliste_aze14',
  [1.9600000381469727,
   3.669999837875366,
   1.380000114440918,
   3.70000004768

# Sommer les délais pour chaque cycliste

In [107]:
import numpy as np
delais_concatenes = delais_par_cyclistes.reduceByKey(concatenation_de_liste)
infos             = delais_concatenes.take(3)
for id_cycliste, durees in infos:
    print ("--------------")
    print (id_cycliste)
    print ("np.min(durees) = %.2f"%np.min(durees))
    print ("np.max(durees) = %.2f"%np.max(durees))
    print ("np.mean(durees)= %.2f"%np.mean(durees))
    print ("np.sum(durees) = %.2f"%np.sum(durees))

--------------
cycliste_aze12
np.min(durees) = 0.00
np.max(durees) = 4.58
np.mean(durees)= 2.36
np.sum(durees) = 96.94
--------------
cycliste_aze14
np.min(durees) = 0.00
np.max(durees) = 4.52
np.mean(durees)= 2.37
np.sum(durees) = 97.06
--------------
cycliste_aze17
np.min(durees) = 0.00
np.max(durees) = 5.25
np.mean(durees)= 2.84
np.sum(durees) = 96.72


In [108]:
delais_concatenes.partitionBy(100)

MapPartitionsRDD[160] at mapPartitions at PythonRDD.scala:422

In [109]:
temps_de_parcours = delais_par_cyclistes.mapValues(lambda x:sum(x))

In [110]:
temps_de_parcours.take(5)

[('cycliste_azrf2', 96.3899998664856),
 ('cycliste_azrf4', 0),
 ('cycliste_azrf5', 98.32999992370605),
 ('cycliste_azrf8', 99.37999987602234),
 ('cycliste_azrfc', 439.7699999809265)]

# ... d'un partitionnement vers le même partitionnement

In [111]:
liste_par_defaut   = []

In [112]:
moyenne_ou_rien = lambda default,liste   : [np.mean(liste)] if liste else default 

duree_moyenne_par_cycliste = delais_concatenes.aggregateByKey(  liste_par_defaut ,
                                                                moyenne_ou_rien  ,
                                                                moyenne_de_liste )

duree_moyenne_par_cycliste.take(3)

[('cycliste_aze12', [2.3643902452980599]),
 ('cycliste_aze14', [2.3673170717751106]),
 ('cycliste_aze17', [2.8447058831944183])]

In [113]:
max_ou_rien = lambda default,liste   : [np.max(liste)] if liste else default 

duree_max_par_cycliste = delais_concatenes.aggregateByKey(  liste_par_defaut ,
                                                                max_ou_rien  ,
                                                                max_de_liste )

duree_max_par_cycliste.take(3)

[('cycliste_azrf2', [4.8600001335144043]),
 ('cycliste_azrf4', []),
 ('cycliste_azrf5', [4.559999942779541])]